In [3]:
from database import Database, LoadDatabase
import numpy as np

In [4]:
DB = LoadDatabase()

In [246]:
class CFRatingPredictor:
    def __init__(self, db):
        self.standings = db.standings
        self.contests = list(db.contests.index)
        self.contests.reverse()
    
    def intDiv(self, x,y):
        return - ((-x) // y) if x < 0 else x // y
    
    def Pij(self, ri, rj):
        return 1.0 / (1.0 + np.power(10.0, (rj-ri)/400.0))
    
    def getSeed(self, rating, contestant, contestStandings):
        prob = lambda r : self.Pij(r, rating)
        vecProb = np.vectorize(prob)
        oldRatings = contestStandings['oldRating'].to_numpy()
        seed = np.sum(vecProb(oldRatings)) - self.Pij(contestant['oldRating'],rating) + 1
        return seed
    
    def getInitialRatingChange(self, contestant, contestStandings):
        rank = contestant['rank']
        oldRating = contestant['oldRating']
        seed = self.getSeed(oldRating, contestant, contestStandings)
        print(contestant.name)
        print(seed)
        midRank = np.sqrt(rank*seed)
        R = self.binarySearch(midRank, lambda r: self.getSeed(r, contestant, contestStandings))
        print(R)
        print(oldRating)
        delta = self.intDiv(R-oldRating,2)
        print(delta)
        return delta
    
    def binarySearch(self, value, f, left=1, right=8000):
        while right - left > 1:
            mid = self.intDiv(left+right,2)
            print(left, right)
            print(mid, f(mid), value)
            if f(mid) < value:
                right = mid
            else:
                left = mid
        return left
     
    def fightAgainstInflation(self, contestStandings, deltas):
        contestantsSorted = contestStandings.sort_values(by=['oldRating'], ascending=False)
        pplCount = len(contestStandings)
        topPplCount = (min(pplCount, 4*(np.rint(np.sqrt(pplCount)))))
        topContestans = contestantsSorted.head(topPplCount)
        sum = np.sum([deltas[i] for i in topContestans.index])
        inc = min(0, max(-10, -(self.intDiv(sum, topPplCount))))
        print(inc)
        return deltas + inc
    
    def fixToSumZero(self, contestStandings, deltas):
        sum = np.sum(deltas)
        inc = -self.intDiv(sum, len(contestStandings)) - 1
        print(inc)
        return deltas + inc
    
    def processContest(self, contestStandings):
        deltas = contestStandings.apply(lambda c: self.getInitialRatingChange(c, contestStandings), axis=1)   
        deltas = self.fixToSumZero(contestStandings, deltas)
        deltas = self.fightAgainstInflation(contestStandings, deltas)
        return deltas
    

In [247]:
rp = CFRatingPredictor(DB)

In [248]:
standings = DB.standings[1299]
deltas = rp.processContest(standings)

Retired_MiFaFaOvO
3.330530633845294
1 8000
4000 1.157797156250326 1.8249741460758544
1 4000
2000 662.6503930156667 1.8249741460758544
2000 4000
3000 28.60607160774059 1.8249741460758544
3000 4000
3500 3.593694319918359 1.8249741460758544
3500 4000
3750 1.6544276129356508 1.8249741460758544
3500 3750
3625 2.315073677376059 1.8249741460758544
3625 3750
3687 1.9319199536373954 1.8249741460758544
3687 3750
3718 1.783437675462393 1.8249741460758544
3687 3718
3702 1.8569402711654641 1.8249741460758544
3702 3718
3710 1.8193861760527188 1.8249741460758544
3702 3710
3706 1.8379583420300651 1.8249741460758544
3706 3710
3708 1.828621567494126 1.8249741460758544
3708 3710
3709 1.8239912644585679 1.8249741460758544
3708
3520
94
TLE
9.590914015019214
1 8000
4000 1.2023224711183689 4.379706386281897
1 4000
2000 662.650898617858 4.379706386281897
2000 4000
3000 28.731991332762355 4.379706386281897
3000 4000
3500 3.9113319520207823 4.379706386281897
3000 3500
3250 10.44765514874957 4.379706386281897
32

3000 4000
3500 4.09862016378994 26.723377845833795
3000 3500
3250 10.884523860414992 26.723377845833795
3000 3250
3125 17.914669654833894 26.723377845833795
3000 3125
3062 22.9655789956548 26.723377845833795
3000 3062
3031 25.928385273708386 26.723377845833795
3000 3031
3015 27.597641322442513 26.723377845833795
3015 3031
3023 26.75053957150601 26.723377845833795
3023 3031
3027 26.33638776675673 26.723377845833795
3023 3027
3025 26.542689577175466 26.723377845833795
3023 3025
3024 26.646420369013022 26.723377845833795
3023
2855
84
ohweonfire
52.78539026331266
1 8000
4000 1.2158466286080793 28.13860078166094
1 4000
2000 662.6578497304055 28.13860078166094
2000 4000
3000 29.266511274987586 28.13860078166094
3000 4000
3500 4.09979580583228 28.13860078166094
3000 3500
3250 10.888815860218818 28.13860078166094
3000 3250
3125 17.922006357733093 28.13860078166094
3000 3125
3062 22.974708652851607 28.13860078166094
3000 3062
3031 25.938386093301904 28.13860078166094
3000 3031
3015 27.608074217

2000 662.6590683297122 38.913239836191565
2000 4000
3000 29.296453919246805 38.913239836191565
2000 3000
2500 180.88326357815077 38.913239836191565
2500 3000
2750 75.4952387614757 38.913239836191565
2750 3000
2875 47.339088160095194 38.913239836191565
2875 3000
2937 37.36505081178239 38.913239836191565
2875 2937
2906 42.07294215873257 38.913239836191565
2906 2937
2921 39.72868262098908 38.913239836191565
2921 2937
2929 38.52964725222083 38.913239836191565
2921 2929
2925 39.124802514966724 38.913239836191565
2925 2929
2927 38.8261415182128 38.913239836191565
2925 2927
2926 38.975200273087154 38.913239836191565
2926
2820
53
ecnerwala
18.17223066025145
1 8000
4000 1.2109847750583729 22.150625901468093
1 4000
2000 662.6518444565162 22.150625901468093
2000 4000
3000 28.896057089275498 22.150625901468093
3000 4000
3500 4.023117919800984 22.150625901468093
3000 3500
3250 10.660410214228245 22.150625901468093
3000 3250
3125 17.600654558690504 22.150625901468093
3000 3125
3062 22.62005732516503

2858 50.54089078852432 50.7018786333431
2857
2780
38
isaf27
40.83235451169931
1 8000
4000 1.2152353040880048 39.905661577729454
1 4000
2000 662.6554254168235 39.905661577729454
2000 4000
3000 29.18099508249365 39.905661577729454
2000 3000
2500 180.83136757848715 39.905661577729454
2500 3000
2750 75.37587872828618 39.905661577729454
2750 3000
2875 47.206122033728775 39.905661577729454
2875 3000
2937 37.23724695907833 39.905661577729454
2875 2937
2906 41.94157298868268 39.905661577729454
2906 2937
2921 39.598803400251256 39.905661577729454
2906 2921
2913 40.83235451169931 39.905661577729454
2913 2921
2917 40.211083447527336 39.905661577729454
2917 2921
2919 39.90382695912869 39.905661577729454
2917 2919
2918 40.05717516335696 39.905661577729454
2918
2913
2
HeHere
366.71106770161845
1 8000
4000 1.217101596478095 121.11334653152285
1 4000
2000 662.8272099225406 121.11334653152285
2000 4000
3000 29.54385183996665 121.11334653152285
2000 3000
2500 181.53915307372333 121.11334653152285
2500 3

2742 2746
2744 77.44009337777628 77.81247390938076
2742 2744
2743 77.72325635473305 77.81247390938076
2742
2630
56
uflbyk
261.7376919778397
1 8000
4000 1.2170569065860373 118.88580809669143
1 4000
2000 662.7490477223407 118.88580809669143
2000 4000
3000 29.53031301955469 118.88580809669143
2000 3000
2500 181.40745578079392 118.88580809669143
2500 3000
2750 75.98620422505833 118.88580809669143
2500 2750
2625 118.79856550949489 118.88580809669143
2500 2625
2562 147.5323510911385 118.88580809669143
2562 2625
2593 132.72027068920139 118.88580809669143
2593 2625
2609 125.59144294976416 118.88580809669143
2609 2625
2617 122.15374792697267 118.88580809669143
2617 2625
2621 120.4659347819942 118.88580809669143
2621 2625
2623 119.62970617154109 118.88580809669143
2623 2625
2624 119.21350128508723 118.88580809669143
2624
2384
120
Lawali
416.1439497271873
1 8000
4000 1.2171128229189025 151.28753165742145
1 4000
2000 662.8710939306429 151.28753165742145
2000 4000
3000 29.547312183618036 151.287531

...

1500 2000
1750 883.8067188360192 829.9821549025637
1750 2000
1875 789.5414330046568 829.9821549025637
1750 1875
1812 841.5125996227817 829.9821549025637
1812 1875
1843 817.0553631576324 829.9821549025637
1812 1843
1827 829.952932858616 829.9821549025637
1812 1827
1819 836.1824510965698 829.9821549025637
1819 1827
1823 833.0860492884077 829.9821549025637
1823 1827
1825 831.5240751986958 829.9821549025637
1825 1827
1826 830.739649352814 829.9821549025637
1826
1955
-64
rafaelgo2
761.8322986113305
1 8000
4000 1.2171423117667064 858.7512241946255
1 4000
2000 663.2836422515336 858.7512241946255
1 2000
1000 1012.831188755913 858.7512241946255
1000 2000
1500 978.4801942427002 858.7512241946255
1500 2000
1750 883.862325468973 858.7512241946255
1750 2000
1875 789.6115033327877 858.7512241946255
1750 1875
1812 841.5767790468951 858.7512241946255
1750 1812
1781 863.8198347711132 858.7512241946255
1781 1812
1796 853.3337445837499 858.7512241946255
1781 1796
1788 858.9907712982443 858.7512241946255


1968 2000
1984 680.4637987485363 690.0797686261469
1968 1984
1976 689.1087779107654 690.0797686261469
1968 1976
1972 693.3996981418051 690.0797686261469
1972 1976
1974 691.2569400831932 690.0797686261469
1974 1976
1975 690.1835292101197 690.0797686261469
1975
2155
-90
CMaster
519.3971553843106
1 8000
4000 1.2171274459781063 714.5399945019014
1 4000
2000 662.9764583635741 714.5399945019014
1 2000
1000 1012.8272833288261 714.5399945019014
1000 2000
1500 978.4181413642312 714.5399945019014
1500 2000
1750 883.6746705731946 714.5399945019014
1750 2000
1875 789.3453618273242 714.5399945019014
1875 2000
1937 729.9676051758457 714.5399945019014
1937 2000
1968 697.7015860113862 714.5399945019014
1937 1968
1952 714.5433325765225 714.5399945019014
1952 1968
1960 706.1703004594186 714.5399945019014
1952 1960
1956 710.3690943361509 714.5399945019014
1952 1956
1954 712.4593219295858 714.5399945019014
1952 1954
1953 713.5021092055857 714.5399945019014
1952
2126
-87
abisheka
685.029089390016
1 8000
40

1750 883.731729404937 786.1310679805889
1750 2000
1875 789.4358129949112 786.1310679805889
1875 2000
1937 730.0742146050861 786.1310679805889
1875 1937
1906 760.6921612407905 786.1310679805889
1875 1906
1890 775.7740676904633 786.1310679805889
1875 1890
1882 783.1191200185276 786.1310679805889
1875 1882
1878 786.74138184394 786.1310679805889
1878 1882
1880 784.9344750610627 786.1310679805889
1878 1880
1879 785.8389870487925 786.1310679805889
1878
2038
-80
Riolku
724.0838325875933
1 8000
4000 1.2171408966274608 849.6538007269963
1 4000
2000 663.2315360298313 849.6538007269963
1 2000
1000 1012.8301257496721 849.6538007269963
1000 2000
1500 978.4640549076297 849.6538007269963
1500 2000
1750 883.8193651059662 849.6538007269963
1750 2000
1875 789.5579406515042 849.6538007269963
1750 1875
1812 841.5274386597741 849.6538007269963
1750 1812
1781 863.7734836287375 849.6538007269963
1781 1812
1796 853.2858872203625 849.6538007269963
1796 1812
1804 847.4804932223861 849.6538007269963
1796 1804
18

1000 2000
1500 978.4584644772729 846.3396102072228
1500 2000
1750 883.8036280964043 846.3396102072228
1750 2000
1875 789.537344793905 846.3396102072228
1750 1875
1812 841.5089507090933 846.3396102072228
1750 1812
1781 863.7563182626125 846.3396102072228
1781 1812
1796 853.268064900433 846.3396102072228
1796 1812
1804 847.4623330413852 846.3396102072228
1804 1812
1808 844.5040937731757 846.3396102072228
1804 1808
1806 845.9878275413923 846.3396102072228
1804 1806
1805 846.7262339162558 846.3396102072228
1805
1958
-76
2om_neek
411.90321778437755
1 8000
4000 1.2171120012548546 645.317095062579
1 4000
2000 662.8671571080528 645.317095062579
2000 4000
3000 29.54705809948143 645.317095062579
2000 3000
2500 181.5777106051996 645.317095062579
2000 2500
2250 383.88015735528205 645.317095062579
2000 2250
2125 520.4046487442647 645.317095062579
2000 2125
2062 592.8221074906556 645.317095062579
2000 2062
2031 628.1785643213358 645.317095062579
2000 2031
2015 646.1883553824811 645.317095062579
2015

TypeError: cannot do positional indexing on Index with these indexers [128.0] of type float64

In [249]:
count = 0
for i in deltas.index:
    if deltas[i] != standings.loc[i]['delta']:
        print("nickname:     ", i)
        print("counted delta:", deltas[i])
        print("actual  delta:", standings.loc[i]['delta'])
        count += 1
print(count)

nickname:      scott_wu
counted delta: -14
actual  delta: -15
nickname:      peehs_moorhsum
counted delta: 71
actual  delta: 69
nickname:      izban
counted delta: 13
actual  delta: 10
nickname:      f.bialas
counted delta: 110
actual  delta: 107
nickname:      AndreySergunin
counted delta: 39
actual  delta: 37
nickname:      zjczzzjczjczzzjc
counted delta: 24
actual  delta: 23
nickname:      pichulia
counted delta: 186
actual  delta: 185
nickname:      Z18
counted delta: 114
actual  delta: 113
nickname:      lumibons
counted delta: 42
actual  delta: 41
nickname:      falanga
counted delta: 145
actual  delta: 144
nickname:      receed
counted delta: 14
actual  delta: 13
nickname:      milisav
counted delta: 95
actual  delta: 94
nickname:      Tlatoani
counted delta: 98
actual  delta: 97
nickname:      pupiI
counted delta: 99
actual  delta: 98
nickname:      ..vince
counted delta: 156
actual  delta: 155
nickname:      Retro3014
counted delta: 80
actual  delta: 79
nickname:      fedoseev

actual  delta: -64
nickname:      Bekh
counted delta: -62
actual  delta: -63
nickname:      arshad2117
counted delta: -79
actual  delta: -80
nickname:      leduythuc
counted delta: -77
actual  delta: -78
nickname:      Hasan0540
counted delta: -106
actual  delta: -107
nickname:      ZeyadKhattab
counted delta: -90
actual  delta: -91
nickname:      QAQmika
counted delta: -65
actual  delta: -67
nickname:      Dormi
counted delta: -78
actual  delta: -79
nickname:      meeeeeeeeendko
counted delta: -63
actual  delta: -64
nickname:      SpargelTarzan
counted delta: -65
actual  delta: -66
nickname:      kazuki08
counted delta: -71
actual  delta: -72
nickname:      vpike
counted delta: -71
actual  delta: -72
nickname:      dixitgarg
counted delta: -71
actual  delta: -72
nickname:      Hemose
counted delta: -63
actual  delta: -64
nickname:      shdut0901
counted delta: -82
actual  delta: -83
nickname:      Shinbai
counted delta: -69
actual  delta: -70
nickname:      TheAnshul
counted delta: -7

In [233]:
standings.head(10)

,rank,oldRating,newRating,delta
handle,,,,
Retired_MiFaFaOvO,1,3520,3600,80
TLE,2,3271,3356,85
Radewoosh,3,3206,3281,75
zhouyuyang,4,2592,2813,221
Benq,5,3274,3299,25
cz_xuyixuan,6,2925,3030,105
maroonrk,7,3011,3084,73
hos.lyric,8,2806,2924,118
yosupo,9,3149,3170,21


In [234]:
deltas.head(10)

handle
Retired_MiFaFaOvO     80
TLE                   85
Radewoosh             75
zhouyuyang           221
Benq                  25
cz_xuyixuan          105
maroonrk              73
hos.lyric            118
yosupo                21
fateice               18
dtype: int64

In [235]:
standings.loc['izban']

rank           46
oldRating    2775
newRating    2785
delta          10
Name: izban, dtype: int64

In [245]:
standings = DB.standings[1299]
rp.getSeed(2821, standings.loc['izban'], standings)

58.086779782455686